In [3]:
import os
import sys
import numpy as np
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser, JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import MessagesPlaceholder
from langchain_community.document_loaders import WebBaseLoader
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS
from langchain.chains import create_retrieval_chain

current_path = os.getcwd()
parrent_path = os.path.abspath(os.path.join(current_path, '..'))
sys.path.append(parrent_path)

from Update_Git import git_add, git_commit, git_push

file_path = os.path.join(current_path, 'Test_5.ipynb')
git_add(file_path)
git_commit('Updated test 5')
git_push('main')

os.environ.pop("OPENAI_API_KEY", None) # Because it loads a key from some place I dont know!
env_path = os.path.join(current_path, ".env")
load_dotenv(dotenv_path=env_path)

True

In [9]:
# Function to load and split documents from a given URL
def document_loader(url):
    loader = WebBaseLoader(url)  # Load the document from the web
    docs = loader.load()  # Extract the content
    
    # Split the document into smaller chunks for processing
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,  # Each chunk has a max size of 200 characters
        chunk_overlap=50  # Overlapping of 50 characters between chunks for context continuity
    )
    split_docs = splitter.split_documents(docs)  # Split the documents

    return split_docs  # Return the processed document chunks

# Function to create a FAISS vector database
def create_db(docs):
    embedding = OpenAIEmbeddings()  # Generate embeddings using OpenAI
    vector_store = FAISS.from_documents(docs, embedding=embedding)  # Store documents in FAISS index
    return vector_store  # Return the FAISS vector store

# Function to create a retrieval chain using the vector store
def create_chain(vector_store):
    model = ChatOpenAI(
        model='gpt-3.5-turbo',  # Use GPT-3.5-turbo model
        temperature=0.4  # Set temperature for response variability
    )

    # Define a prompt template for the chatbot
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Answer the user's question based on the given context: {context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}")
    ])

    parser = StrOutputParser()  # Convert output to a string format

    # Create the document processing chain
    chain = create_stuff_documents_chain(
        llm=model,
        prompt=prompt,
        output_parser=parser
    )

    retriever = vector_store.as_retriever(search_kwaargs = {"k": 3})  # Convert vector store into a retriever
    retrieval_chain = create_retrieval_chain(
        retriever, 
        chain
        )  # Create a retrieval-based chain

    return retrieval_chain  # Return the retrieval chain


def process_chat(chain, question, history):
    # Query the retrieval chain with a sample question
    response = chain.invoke({
    "input": question,
    "chat_history": history,
    "context": docs
})

    # Print the answer from the model
    return response["answer"]


if __name__ == "__main__":
    docs = document_loader("https://en.wikipedia.org/wiki/FreeSurfer")
    vector_store = create_db(docs)
    chain = create_chain(vector_store)

    chat_history = []

    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            break
        response = process_chat(chain, user_input, chat_history)
        chat_history.append(HumanMessage(content=user_input))
        chat_history.append(AIMessage(content=response))
        print("AI: ", response)





AI:  Hello! How can I assist you today?
AI:  Based on the provided information, the document seems to be related to a research paper or article in the field of neuroscience or neuroimaging. It includes authors such as Stephen M. Smith, Karla L. Miller, and Paul M. Matthews, among others. The document likely discusses topics related to brain imaging, diffusion MRI, and possibly methods for analyzing brain connectivity.
AI:  Yes, the document appears to be related to a toolbox called FreeSurfer. FreeSurfer is a software suite used for analyzing and visualizing structural and functional neuroimaging data. It is commonly used in the field of neuroscience for tasks such as brain segmentation, cortical surface reconstruction, and volumetric analysis. The document likely provides information on the usage, interoperation, and download of the FreeSurfer toolbox.
AI:  In addition to the usage and interoperation of the FreeSurfer toolbox, the document may also cover topics such as the various pac